# Collecting Tweets using Twitter Academic API

## 1. Import Packages & Define Key Variables

In [1]:
# Import packages
import yaml
from searchtweets import ResultStream, gen_request_parameters, load_credentials, collect_results

# Define rootpath
rp = 'C:\\Users\\sgmmahon\\Documents\\GitHub\\iom_project\\'
mp = 'methods\\accessing_tweets\\'
dp = 'data\\tweet_data\\'   

In [2]:
# Define YAML with key details for accessing Twitter API
config = dict(
    search_tweets_v2=dict(
        endpoint=f"https://api.twitter.com/2/tweets/search/all",
        consumer_key='n5q87cELjVKlO7Z8MXXjp0Chr',
        consumer_secret='Tg3bEINKtvvxXpD5HwyJ2bCtd99RQl8aeJXMEma1SsE6Q3zLXa',
        bearer_token='AAAAAAAAAAAAAAAAAAAAABLt8wAAAAAAbE8U4tJJtQffIvLTtuHWVzm0nqI%3DIinQPSHHGhYjXwG71L4AHkUnUmlP9X8npfKQ0S6Nk07nBSF8AY'
    )
)


# Save YAML file
with open(rp + mp + 'twitter_keys.yaml', 'w') as config_file:
    yaml.dump(config, config_file, default_flow_style=False)

In [3]:
# Define rules for academic search for streaming tweets
academic_search_args = load_credentials(filename=rp + mp + 'twitter_keys.yaml',
                                        yaml_key="search_tweets_v2",
                                        env_overwrite=False)

## 2. Define Search Terms

In [4]:
# Create functions which concatenates vectors
def cnct (x): return(" OR ".join(x))
def cnctwb (x): return("(" + " OR ".join(x) + ")")

In [5]:
# UK Search Terms# UK Search Terms

# Migrant terms
uk_neutral_migrant_terms  = ["immigrant", "immigration", "migrant", "migration", "\"asylum seeker\"", "refugee", "\"undocumented worker\"", "\"guest worker\"", 
                             "\"EU worker\"", "\"non-UK workers\"", "\"foreign worker\"", "(human smuggling)", "(human trafficking)"]
uk_negative_migrant_terms = ["illegals", "foreigner", "\"illegal alien\"", "\"illegal worker\""]
 
# Racial terms
uk_negative_racial_terms  = ["islamophob", "sinophob", "\"china flu\"", "\"kung flu\"", "\"china virus\"", "\"chinese virus\"", "shangainese"]

# Twitter accounts
uk_pro_migrant_account_1  = ["@UNmigration", "@IOM_UN", "@IOMatUN", "@IOMatEU", "@IOM_UK", "@IOMResearch", "@IOM_GMDAC", "@hrw", "@Right_to_Remain",
                             "@CommonsHomeAffs", "@fcukba", "@Mark_George_QC", "@MigrantVoiceUK", "@MigrantChildren", "@MigrantHelp", "@thevoiceofdws"]
uk_pro_migrant_account_2  = ["@WORCrights", "@UbuntuGlasgow", "@MigrantsUnionUK", "@migrants_rights", "@MigrantsMRC", "@Consenant_UK", "@RomaSupport",
                             "@MigrantsLawProj", "@MigRightsScot", "@IRMOLondon", "@HighlySkilledUK", "@WeBelong19", "@Project17UK"]
uk_neutral_account        = ["@ukhomeoffice", "@pritipatel", "@UKHomeSecretary", "@EUHomeAffairs", "@MigrMatters", "@MigObs"]
uk_anti_migrant_account   = ["@Nigel_Farage", "@MigrationWatch"]

# Hashtags
uk_positive_hashtags      = ["#RefugeesWelcome", "#MigrantsWelcome", "#LeaveNoOneBehind", "#FreedomForImmigrants", "#illegalmigantsUK", "#LondonIsOpen",
                             "#EndHostileEnvironment", "#FamiliesBelongTogether"]
uk_neutral_hashtags       = ["#Pritiuseless", "#migrationEU", "#immigration", "#migration", "#immigrant", "#migrant", "#immigrate", "#migrate", "#refugees",
                             "#NigelFarage", "#ImmigrationReform"]
uk_negative_hashtags      = ["#illegals", "#foreigner", "#foreigners", "#illegalalien", "#illegalaliens", "#illegalworker", "#illegalworkers", "#KeepThemOut",
                             "#OurCountry", "#SendThemBack", "#migrantsnotwelcome", "#refugeesnotwelcome", "#illegals", "#ChinaVirus", "#chinaflu", "#kungflu",
                             "#chinesevirus", "#TheyHaveToGoBack", "#DeportThemAll"]
uk_event_hashtags         = ["#Moria", "#CampFire", "#closethecamps"]

# Define final search queries
uk_terms    = cnctwb([cnct(uk_neutral_migrant_terms), cnct(uk_negative_migrant_terms), cnct(uk_negative_racial_terms)])
uk_accounts = cnctwb([cnct(uk_pro_migrant_account_1), cnct(uk_pro_migrant_account_2), cnct(uk_neutral_account), cnct(uk_anti_migrant_account)])
uk_hashtags = cnctwb([cnct(uk_positive_hashtags), cnct(uk_neutral_hashtags), cnct(uk_negative_hashtags), cnct(uk_event_hashtags)])

# Append all search term into single list
uk_search_terms = uk_neutral_migrant_terms + uk_negative_migrant_terms + uk_negative_racial_terms + uk_pro_migrant_account_1 + \
                  uk_pro_migrant_account_2 + uk_neutral_account + uk_positive_hashtags + uk_neutral_hashtags + uk_negative_hashtags + uk_event_hashtags


# USA Search Terms

# Migrant terms
usa_neutral_migrant_terms  = ["immigrant", "immigration", "migrant", "migration", "\"asylum seeker\"", "refugee", "\"undocumented worker\"", "\"guest worker\"", 
                              "\"foreign worker\"", "(human smuggling)", "(human trafficking)"]
usa_negative_migrant_terms = ["illegals", "foreigner", "\"illegal alien\"", "\"illegal worker\""]
 
# Racial terms
usa_negative_racial_terms  = ["islamophob", "sinophob", "\"china flu\"", "\"kung flu\"", "\"china virus\"", "\"chinese virus\"", "shangainese"]

# Twitter accounts
usa_pro_migrant_account_1  = ["@UNmigration", "@IOM_UN", "@IOMatUN", "@IOMatEU", "@IOM_UK", "@IOMResearch", "@IOM_GMDAC", "@hrw", "@NIJC", "@CIYJA", 
                              "@ImmAdvocates", "@NWIRP", "@RAICESTEXAS", "@ImmJusticeNOW", "@icirr", "@IAmAnImmigrant", "@Am4ImmJustice", "@NILCJusticeFund"]
usa_pro_migrant_account_2  = ["@CIRCimmigrant", "@FLImmigrant", "@ImmFamTogether", "@ImmJustice", "@NICE4Workers", "@CA4ImmiJustice", "@immigrantarc", "@Join_SIM",
                              "@SDIRC", "@RMIAN_org", "@NJAIJ", "@NVImmigrants", "@VA_Immigrants"]
usa_neutral_account        = ["@ICEgov", "@PhillyOIA", "@iandraffairs", "@LAC4Immigrants", "@CoreCivic"]
usa_anti_migrant_account   = ["@EuropidWhites"]

# Hashtags
usa_positive_hashtags      = ["#RefugeesWelcome", "#MigrantsWelcome", "#LeaveNoOneBehind", "#FreedomForImmigrants", "#illegalmigantsUSA", "#KillTheImmigrationBill", 
                              "#ImmigrantsMakeAmericaGreat", "#NoWall", "#NoWallEver", "#NoBan", "#FamiliesBelongTogether", "#stopICEcold", "#EndRemainInMexico"]
usa_neutral_hashtags       = ["#ICE", "#immigration", "#migration", "#immigrant", "#migrant", "#immigrate", "#migrate", "#refugees", "#ImmigrationReform"]
usa_negative_hashtags      = ["#illegals", "#foreigner", "#foreigners", "#illegalalien", "#illegalaliens", "#illegalworker", "#illegalworkers", "#KeepThemOut",
                              "#OurCountry", "#SendThemBack", "#migrantsnotwelcome", "#refugeesnotwelcome", "#illegals", "#ChinaVirus", "#chinaflu", "#kungflu",
                              "#chinesevirus", "#TheyHaveToGoBack", "#DeportThemAll"]
usa_event_hashtags         = ["#Moria", "#closethecamps", "#divestfromdetention"]

# Define final search queries
usa_terms    = cnctwb([cnct(usa_neutral_migrant_terms), cnct(usa_negative_migrant_terms), cnct(usa_negative_racial_terms)])
usa_accounts = cnctwb([cnct(usa_pro_migrant_account_1), cnct(usa_pro_migrant_account_2), cnct(usa_neutral_account), cnct(usa_anti_migrant_account)])
usa_hashtags = cnctwb([cnct(usa_positive_hashtags), cnct(usa_neutral_hashtags), cnct(usa_negative_hashtags), cnct(usa_event_hashtags)])

# Append all search term into single list
usa_search_terms = usa_neutral_migrant_terms + usa_negative_migrant_terms + usa_negative_racial_terms + usa_pro_migrant_account_1 + \
                   usa_pro_migrant_account_2 + usa_neutral_account + usa_positive_hashtags + usa_neutral_hashtags + usa_negative_hashtags + usa_event_hashtags



# Country Search Terms
uk_add_terms     = ' lang:en place_country:GB'
usa_add_terms    = ' lang:en (place_country:US OR profile_country:US)'

## 3. Search Tweets

In [59]:
help(gen_request_parameters)

Help on function gen_request_parameters in module searchtweets.api_utils:

gen_request_parameters(query, granularity, results_per_call=None, start_time=None, end_time=None, since_id=None, until_id=None, tweet_fields=None, user_fields=None, media_fields=None, place_fields=None, poll_fields=None, expansions=None, stringify=True)
    Generates the dict or json payload for a search query.
    
    Args:
        query (str): The string version of a search query,
            e.g., "snow has:media -is:retweet". Accepts multi-line strings
            for ease of entry.
        results_per_call (int): number of tweets or counts returned per API
        call. This maps to the `max_results`` search API parameter.
            Defaults to 100 (maximum supported in Labs).
        start_time (str or None): Date format as specified by
            `convert_utc_time` for the starting time of your search.
        end_time (str or None): date format as specified by `convert_utc_time`
            for the e

In [33]:
query = gen_request_parameters(uk_terms + uk_add_terms, results_per_call=100, granularity = "hour", start_time = "2020-02-01T00:00")
query

'{"query":"(immigrant OR immigration OR migrant OR migration OR \\"asylum seeker\\" OR refugee OR \\"undocumented worker\\" OR \\"guest worker\\" OR \\"EU worker\\" OR \\"non-UK workers\\" OR \\"foreign worker\\" OR (human smuggling) OR (human trafficking) OR illegals OR foreigner OR \\"illegal alien\\" OR \\"illegal worker\\" OR islamophob OR sinophob OR \\"china flu\\" OR \\"kung flu\\" OR \\"china virus\\" OR \\"chinese virus\\" OR shangainese) lang:en place_country:GB","start_time":"2020-02-01T00:00:00Z","granularity":"hour"}'

In [28]:
help(gen_request_parameters)

Help on function gen_request_parameters in module searchtweets.api_utils:

gen_request_parameters(query, granularity, results_per_call=None, start_time=None, end_time=None, since_id=None, until_id=None, tweet_fields=None, user_fields=None, media_fields=None, place_fields=None, poll_fields=None, expansions=None, stringify=True)
    Generates the dict or json payload for a search query.
    
    Args:
        query (str): The string version of a search query,
            e.g., "snow has:media -is:retweet". Accepts multi-line strings
            for ease of entry.
        results_per_call (int): number of tweets or counts returned per API
        call. This maps to the `max_results`` search API parameter.
            Defaults to 100 (maximum supported in Labs).
        start_time (str or None): Date format as specified by
            `convert_utc_time` for the starting time of your search.
        end_time (str or None): date format as specified by `convert_utc_time`
            for the e

In [9]:
# Query inputs
search_terms = uk_terms
add_terms = uk_add_terms
max_results = 100
granularity = "hour"
start_time  = '\"2020-02-01T00:00:00Z\"'
end_time    = '\"2020-02-08T00:00:00Z\"'

# Build queries
tweet_full_query  = '{"query":' + '"' + search_terms.replace('"', '\\"') + add_terms.replace('"', '\\"') + '"' + ',"max_results":' + str(max_results) + \
                    ',"start_time":' + start_time + ',"end_time":' + end_time + '}'
tweet_count_query = '{"query":' + '"' + search_terms.replace('"', '\\"') + add_terms.replace('"', '\\"') + '"' + ',"granulalarity":' + granularity + '}'
tweet_full_query

'{"query":"(immigrant OR immigration OR migrant OR migration OR \\"asylum seeker\\" OR refugee OR \\"undocumented worker\\" OR \\"guest worker\\" OR \\"EU worker\\" OR \\"non-UK workers\\" OR \\"foreign worker\\" OR (human smuggling) OR (human trafficking) OR illegals OR foreigner OR \\"illegal alien\\" OR \\"illegal worker\\" OR islamophob OR sinophob OR \\"china flu\\" OR \\"kung flu\\" OR \\"china virus\\" OR \\"chinese virus\\" OR shangainese) lang:en place_country:GB","max_results":100,"start_time":"2020-02-01T00:00:00Z","end_time":"2020-02-08T00:00:00Z"}'

In [25]:
tweets = collect_results(tweet_full_query,
                         max_tweets=100,
                         result_stream_args=academic_search_args)

In [85]:
help(collect_results)

Help on function collect_results in module searchtweets.result_stream:

collect_results(query, max_tweets=1000, result_stream_args=None)
    Utility function to quickly get a list of tweets from a ``ResultStream``
    without keeping the object around. Requires your args to be configured
    prior to using.
    Args:
        query (str): valid powertrack rule for your account, preferably
        generated by the `gen_request_parameters` function.
        max_tweets (int): maximum number of tweets or counts to return from
        the API / underlying ``ResultStream`` object.
        result_stream_args (dict): configuration dict that has connection
        information for a ``ResultStream`` object.
    Returns:
        list of results
    Example:
        >>> from searchtweets import collect_results
        >>> tweets = collect_results(query,
                                     max_tweets=500,
                                     result_stream_args=search_args)



In [39]:
rs = ResultStream(request_parameters=tweet_full_query,
                  max_results=500,
                  max_pages=1,
                  **academic_search_args)

In [41]:
tweets = rs.stream()
tweets

<bound method ResultStream.stream of ResultStream: 
	{
    "endpoint":"https:\/\/api.twitter.com\/2\/tweets\/search\/all",
    "request_parameters":{
        "query":"(immigrant OR immigration OR migrant OR migration OR \"asylum seeker\" OR refugee OR \"undocumented worker\" OR \"guest worker\" OR \"EU worker\" OR \"non-UK workers\" OR \"foreign worker\" OR (human smuggling) OR (human trafficking) OR illegals OR foreigner OR \"illegal alien\" OR \"illegal worker\" OR islamophob OR sinophob OR \"china flu\" OR \"kung flu\" OR \"china virus\" OR \"chinese virus\" OR shangainese) lang:en place_country:GB",
        "max_results":100,
        "start_time":"2020-02-01T00:00:00Z",
        "end_time":"2020-02-08T00:00:00Z"
    },
    "max_tweets":500
}>

In [42]:
tweets = list(rs.stream())

In [49]:
tweets

[{'data': [{'id': '1225922769210179585',
    'text': '@itvnews If accessible housing stock is being taken up by Eastern Europeans &amp; illegal immigrants thanks to the EU immigration policy, why is it such a surprise that British #DanielSlade can’t find a home in his own nation?! cc @BBCNews @SkyNews'},
   {'id': '1225920400414191617',
    'text': '@redsarah99 I apologise for using the word Immigrant I Ment to say Refugee'},
   {'id': '1225919615185846273',
    'text': 'So Mr B do you as a good Labour voter not agree with immigration at this Level ? Maybe you should look for a new party ! To follow https://t.co/66aCsQK3KQ'},
   {'id': '1225919526853804035',
    'text': '@dmoss61 Yeah and we need to resist the pressure to rip up all our green spaces to build houses for mass immigration'},
   {'id': '1225909383105826816',
    'text': "@Bonn1eGreer It should be, if not, where does this end? My own belief is that her biggest crime was to be poor. As she would've been able to make her way 

In [7]:
help(ResultStream)

Help on class ResultStream in module searchtweets.result_stream:

class ResultStream(builtins.object)
 |  ResultStream(endpoint, request_parameters, bearer_token=None, extra_headers_dict=None, max_tweets=500, max_requests=None, output_format='r', **kwargs)
 |  
 |  Class to represent an API query that handles two major functionality
 |  pieces: wrapping metadata around a specific API call and automatic
 |  pagination of results.
 |  Args:
 |      bearer_token (str): bearer token for v2.
 |  
 |      endpoint (str): API endpoint.
 |  
 |      request_parameters (json or dict): payload for the post request
 |  
 |      max_tweets (int): max number results that will be returned from this
 |      instance. Note that this can be slightly lower than the total returned
 |      from the API call  - e.g., setting ``max_tweets = 10`` would return
 |      ten results, but an API call will return at minimum 100 results by default.
 |  
 |      max_requests (int): A hard cutoff for the number of AP